In [1]:
#!/usr/bin/env python
# coding: utf-8

# # 2D Advection-Diffusion equation

# in this notebook we provide a simple example of the DeepMoD algorithm and apply it on the 2D advection-diffusion equation.

In [1]:
# General imports
import numpy as np
import torch
import matplotlib.pylab as plt

# DeepMoD functions

from deepymod import DeepMoD
from deepymod.data import Dataset, get_train_test_loader
from deepymod.data.samples import Subsample_random
from deepymod.model.func_approx import NN
from deepymod.model.library import Library2D
from deepymod.model.constraint import LeastSquares
from deepymod.model.sparse_estimators import Threshold, PDEFIND
from deepymod.training import train
from deepymod.training.sparsity_scheduler import TrainTestPeriodic
from scipy.io import loadmat

# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(0)

# Configuring GPU or CPU
#if torch.cuda.is_available():
#    device = "cuda"
#else:
device = "cpu"
print(device)

cpu


## Prepare the data

Next, we prepare the dataset. In order to do so, we write a 
function that loads the data from the matlab file, reshapes it into a shape 
we desire and then return it in a coordinate, data format. Where

In [2]:
def create_data():
    data = loadmat("data/advection_diffusion.mat")
    usol = np.real(data["Expression1"]).astype("float32")
    usol = usol.reshape((51, 51, 61, 4))
    x_v= usol[:,:,:,0]
    y_v = usol[:,:,:,1]
    t_v = usol[:,:,:,2]
    u_v = usol[:,:,:,3]
    coords = torch.from_numpy(np.transpose((t_v.flatten(),x_v.flatten(), y_v.flatten())))
    data = torch.from_numpy(usol[:, :, :, 3].reshape(-1, 1))
    print("The raw data has shape {}".format(data.shape))
    return coords, data



Then we pass this function to the dataset class, which then adds noise to it, normalizes the coordinates and performs random subsampling to it.

In [3]:
dataset = Dataset(
    create_data,
    preprocess_kwargs={
        "noise_level": 0.1,
        "normalize_coords": True,
        "normalize_data": True,
    },
    subsampler=Subsample_random,
    subsampler_kwargs={"number_of_samples": 2500},
    device=device,
)

The raw data has shape torch.Size([158661, 1])
Dataset is using device:  cpu


Next we plot the dataset for three different time-points


In [4]:
train_dataloader, test_dataloader = get_train_test_loader(dataset, train_test_split=0.8)

## Configuration of DeepMoD

Configuration of the function approximator: Here the first argument is the number of input and the last argument the number of output layers.


In [5]:
network = NN(3, [50, 50, 50, 50], 1)

Configuration of the library function: We select athe library with a 2D spatial input. Note that that the max differential order has been pre-determined here out of convinience. So, for poly_order 1 the library contains the following 12 terms:
* [$1, u_x, u_y, u_{xx}, u_{yy}, u_{xy}, u, u u_x, u u_y, u u_{xx}, u u_{yy}, u u_{xy}$]

In [6]:
library = Library2D(poly_order=1)

Configuration of the sparsity estimator and sparsity scheduler used. In this case we use the most basic threshold-based Lasso estimator and a scheduler that asseses the validation loss after a given patience. If that value is smaller than 1e-5, the algorithm is converged.

In [7]:
estimator = Threshold(0.1)
sparsity_scheduler = TrainTestPeriodic()

Configuration of the sparsity estimator

In [8]:
constraint = LeastSquares()


Now we instantiate the model and select the optimizer

In [9]:
model = DeepMoD(network, library, estimator, constraint).to(device)

In [10]:
# Defining optimizer
optimizer = torch.optim.Adam(
    model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=2e-3
)

## Run DeepMoD

We can now run DeepMoD using all the options we have set and the training data:
* The directory where the tensorboard file is written (log_dir)
* The ratio of train/test set used (split)
* The maximum number of iterations performed (max_iterations)
* The absolute change in L1 norm considered converged (delta)
* The amount of epochs over which the absolute change in L1 norm is calculated (patience)

In [11]:
train(
    model,
    train_dataloader,
    test_dataloader,
    optimizer,
    sparsity_scheduler,
    log_dir="runs/2DAD/",
    max_iterations=25000,
    delta = 10**-5
)

 24975  MSE: 2.79e-04  Reg: 1.20e-05  L1: 1.86e+00 Algorithm converged. Writing model to disk.


Sparsity masks provide the active and non-active terms in the PDE:

In [12]:
model.sparsity_masks

[tensor([False,  True,  True,  True,  True, False, False, False, False, False,
         False, False])]

estimator_coeffs gives the magnitude of the active terms:

In [13]:
model.estimator_coeffs()

[array([[0.        ],
        [0.3539985 ],
        [0.71517855],
        [0.3977614 ],
        [0.38704312],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ]], dtype=float32)]